In [20]:
from biollm.repo.gears import GEARS, PertData

Load data. We use norman as an example.

In [22]:
import biollm
biollm.repo.gears.__file__

'/home/share/huadjyin/home/s_qiuping1/workspace/omics_model/bio_model/biollm/biollm/repo/gears/__init__.py'

In [23]:
data_dir = '/home/share/huadjyin/home/s_qiuping1/workspace/omics_model/data/finetune/gears_data/norman'

In [24]:
pert_data = PertData(data_dir)
pert_data.load(data_name = 'norman')
pert_data.prepare_split(split = 'simulation', seed = 1)
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128)

Found local copy...
Found local copy...
the adata shape:
(91205, 5045)
Found local copy...
These perturbations are not in the GO graph and their perturbation can thus not be predicted
['RHOXF2BB+ctrl' 'LYL1+IER5L' 'ctrl+IER5L' 'KIAA1804+ctrl' 'IER5L+ctrl'
 'RHOXF2BB+ZBTB25' 'RHOXF2BB+SET']
Local copy of pyg dataset is detected. Loading...
Done!
Creating new splits....
Saving new splits at /home/share/huadjyin/home/s_qiuping1/workspace/omics_model/data/finetune/gears_data/norman/norman/splits/norman_simulation_1_0.75.pkl
Simulation split test composition:
combo_seen0:9
combo_seen1:43
combo_seen2:19
unseen_single:36
Done!
Creating dataloaders....
Done!


Create a model object; if you use [wandb](https://wandb.ai), you can easily track model training and evaluation by setting `weight_bias_track` to true, and specify the `proj_name` and `exp_name` that you like.

In [25]:
gears_model = GEARS(pert_data, device = 'cuda:0', 
                        weight_bias_track = False, 
                        proj_name = 'pertnet', 
                        exp_name = 'pertnet')
gears_model.model_initialize(hidden_size = 64)

Found local copy...


You can find available tunable parameters in model_initialize via

In [26]:
gears_model.tunable_parameters()

{'hidden_size': 'hidden dimension, default 64',
 'num_go_gnn_layers': 'number of GNN layers for GO graph, default 1',
 'num_gene_gnn_layers': 'number of GNN layers for co-expression gene graph, default 1',
 'decoder_hidden_size': 'hidden dimension for gene-specific decoder, default 16',
 'num_similar_genes_go_graph': 'number of maximum similar K genes in the GO graph, default 20',
 'num_similar_genes_co_express_graph': 'number of maximum similar K genes in the co expression graph, default 20',
 'coexpress_threshold': 'pearson correlation threshold when constructing coexpression graph, default 0.4',
 'uncertainty': 'whether or not to turn on uncertainty mode, default False',
 'uncertainty_reg': 'regularization term to balance uncertainty loss and prediction loss, default 1',
 'direction_lambda': 'regularization term to balance direction loss and prediction loss, default 1'}

Train your model:

Note: For the sake of demo, we set epoch size to 1. To get full model, set `epochs = 20`.

In [27]:
gears_model.train(epochs = 1, lr = 1e-3)

Start Training...


RuntimeError: The size of tensor a (161440) must match the size of tensor b (143552) at non-singleton dimension 0

Save and load pretrained models:

In [7]:
gears_model.save_model('test_model')
gears_model.load_pretrained('test_model')

Make prediction for new perturbation:

In [8]:
gears_model.predict([['FEV'], ['FEV', 'AHR']])

{'FEV': array([-4.3144148e-11,  3.8635105e-02,  2.0985277e-01, ...,
         2.9860971e+00, -2.1779465e-03, -1.4885039e-17], dtype=float32),
 'FEV_AHR': array([-4.9785936e-11,  3.9381046e-02,  2.6676694e-01, ...,
         2.7445843e+00, -5.0356295e-03, -1.8939207e-17], dtype=float32)}

Gene list can be found here:

In [9]:
gears_model.gene_list[:5]

['RP11-34P13.8', 'RP11-54O7.3', 'SAMD11', 'PERM1', 'HES4']

In [19]:
gears_model.model_initialize()

<method '__dir__' of 'object' objects>